In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ModerateCNN(nn.Module):
    def __init__(self):
        super(ModerateCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),
            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            # nn.Linear(4096, 1024),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            # nn.Linear(1024, 512),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

In [15]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class CustomCIFAR10Dataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform

        self.class_names = sorted(os.listdir(data_folder))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        self.image_paths = []
        self.labels = []
        self.data = []

        for class_name in self.class_names:
            class_folder = os.path.join(data_folder, class_name)
            class_label = self.class_to_idx[class_name]
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.image_paths.append(img_path)
                self.labels.append(class_label)
                image = Image.open(img_path)
                self.data.append(np.array(image)) 

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [24]:
from torchvision import transforms

transform_valid = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                    std=[x / 255.0 for x in [63.0, 62.1, 66.7]],
                ),
            ]
        )

In [25]:
model = ModerateCNN()

model_data = './best_local_model_30_epoch.pt' # path del best_local_model.pt

state_dict = torch.load(model_data)
model.load_state_dict(state_dict['model_weights'])
model.eval()

ModerateCNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Dropout2d(p=0.05, inplace=False)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (

In [37]:
from torch.utils.data import DataLoader

test_data_folder = "./CIFAR-10-images/test/"
test_dataset = CustomCIFAR10Dataset(test_data_folder, transform=transform_valid)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        if i >= 1000:  # Break after processing 1000 images
            break
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        predicted_label = predicted.item()
        predicted_class_name = test_dataset.class_names[predicted_label]
        true_label = labels.item()
        true_class_name = test_dataset.class_names[true_label]

        print(f"predicted: {predicted_label} - {predicted_class_name}, actual: {true_label} - {true_class_name}")

        total_predictions += 1
        if predicted_label == true_label:
            correct_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"\nAccuracy: {accuracy*100:.2f}%")



predicted: 4 - deer, actual: 4 - deer
predicted: 5 - dog, actual: 4 - deer
predicted: 3 - cat, actual: 7 - horse
predicted: 2 - bird, actual: 2 - bird
predicted: 0 - airplane, actual: 0 - airplane
predicted: 3 - cat, actual: 3 - cat
predicted: 2 - bird, actual: 7 - horse
predicted: 3 - cat, actual: 0 - airplane
predicted: 4 - deer, actual: 2 - bird
predicted: 8 - ship, actual: 8 - ship
predicted: 6 - frog, actual: 3 - cat
predicted: 6 - frog, actual: 6 - frog
predicted: 0 - airplane, actual: 0 - airplane
predicted: 4 - deer, actual: 4 - deer
predicted: 1 - automobile, actual: 1 - automobile
predicted: 2 - bird, actual: 4 - deer
predicted: 3 - cat, actual: 4 - deer
predicted: 6 - frog, actual: 6 - frog
predicted: 7 - horse, actual: 7 - horse
predicted: 2 - bird, actual: 2 - bird
predicted: 9 - truck, actual: 1 - automobile
predicted: 1 - automobile, actual: 1 - automobile
predicted: 8 - ship, actual: 8 - ship
predicted: 9 - truck, actual: 8 - ship
predicted: 4 - deer, actual: 4 - deer
p